In [182]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [200]:
train_files = pd.read_csv("train.csv")
test_files = pd.read_csv("test.csv")

files = train_files

train_files

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [201]:
# We don't need to know the PassengerID nor the name
files = files.drop(["PassengerId","Name"],axis=1)

In [202]:
# Check how many fields are Null in order to see what we can fill and what we have
files.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [203]:
files["TotalSpent"] = files["RoomService"] + files["FoodCourt"] + files["ShoppingMall"] + files["Spa"] + files["VRDeck"]
files[['Deck','Num','Side']] = files["Cabin"].str.split('/',expand=True)
files = files.drop(["RoomService","FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Cabin"],axis=1)
files

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,TotalSpent,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,False,0.0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,True,736.0,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,False,10383.0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,False,5176.0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,True,1091.0,F,1,S
...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,False,8536.0,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,False,0.0,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,True,1873.0,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,False,4637.0,E,608,S


In [204]:
files.isna().sum()

HomePlanet     201
CryoSleep      217
Destination    182
Age            179
VIP            203
Transported      0
TotalSpent     908
Deck           199
Num            199
Side           199
dtype: int64

In [205]:
#You can«t spend money on cryosleep
files.loc[files["CryoSleep"].isin([True]), "TotalSpent"] = 0
files.loc[files["TotalSpent"].isin([0]), "CryoSleep"] = True


In [206]:
files.isna().sum()

HomePlanet     201
CryoSleep      130
Destination    182
Age            179
VIP            203
Transported      0
TotalSpent     561
Deck           199
Num            199
Side           199
dtype: int64

In [207]:
files["HomePlanet"].fillna(files["HomePlanet"].mode()[0], inplace=True)
files["CryoSleep"].fillna(files["CryoSleep"].mode()[0], inplace=True)
files["Destination"].fillna(files["Destination"].mode()[0], inplace=True)
files["HomePlanet"].fillna(files["HomePlanet"].mode()[0], inplace=True)
files["Age"].fillna(files["Age"].mean(), inplace=True)
files["VIP"].fillna(files["VIP"].mode()[0], inplace=True)
files["TotalSpent"].fillna(files["TotalSpent"].mode()[0], inplace=True)
# files = files.dropna(inplace=True)
# 

In [208]:
files.isna().sum()

HomePlanet       0
CryoSleep        0
Destination      0
Age              0
VIP              0
Transported      0
TotalSpent       0
Deck           199
Num            199
Side           199
dtype: int64

In [209]:
files.dropna(inplace=True)

In [197]:
files

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,TotalSpent,Deck,Num,Side
0,Europa,True,TRAPPIST-1e,39.0,False,False,0.0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,True,736.0,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,False,10383.0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,False,5176.0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,True,1091.0,F,1,S
...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,False,8536.0,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,False,0.0,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,True,1873.0,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,False,4637.0,E,608,S


In [211]:
for column in files.columns:
    print(f"Column: {column}, number of values: {files[column].value_counts().count()}")

Column: HomePlanet, number of values: 3
Column: CryoSleep, number of values: 2
Column: Destination, number of values: 3
Column: Age, number of values: 81
Column: VIP, number of values: 2
Column: Transported, number of values: 2
Column: TotalSpent, number of values: 2081
Column: Deck, number of values: 8
Column: Num, number of values: 1817
Column: Side, number of values: 2


In [212]:
categoricals = files.select_dtypes(exclude=[np.number])
categoricals.describe()

,HomePlanet,CryoSleep,Destination,VIP,Transported,Deck,Num,Side
count,8494,8494,8494,8494,8494,8494,8494,8494
unique,3,2,3,2,2,8,1817,2
top,Earth,False,TRAPPIST-1e,False,True,F,82,S
freq,4702,4997,5966,8301,4278,2794,28,4288


In [213]:
for column in categoricals.columns:
    print(column)
    unique_string = {}
    # new_column = df
    unique_int = 0
    for x in files[column]:
        if x not in unique_string:
            unique_string[x] = unique_int
            unique_int += 1

    files[column] = files[column].replace(unique_string)
            
        

HomePlanet
CryoSleep
Destination
VIP
Transported
Deck
Num
Side


In [214]:
files

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,TotalSpent,Deck,Num,Side
0,0,0,0,39.0,0,0,0.0,0,0,0
1,1,1,0,24.0,0,1,736.0,1,0,1
2,0,1,0,58.0,1,0,10383.0,2,0,1
3,0,1,0,33.0,0,0,5176.0,2,0,1
4,1,1,0,16.0,0,1,1091.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
8688,0,1,2,41.0,1,0,8536.0,2,87,0
8689,1,0,1,18.0,0,0,0.0,3,1422,1
8690,1,1,0,26.0,0,1,1873.0,3,1533,1
8691,0,1,2,32.0,0,0,4637.0,4,727,1


In [215]:
X, y = files.values[:, :-1], files.values[:, -1]
print(X.shape, y.shape)

(8494, 9) (8494,)


In [17]:
model = LogisticRegression(solver='liblinear')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [19]:
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.789 (0.016)
